In [1162]:
import pandas as pd, numpy as np, sqlite3 as sql, datetime as dt, re, time, yfinance as yf
from dateutil.relativedelta import relativedelta

In [1174]:
with sql.connect('../data/interim/companies.db') as con:
        port = pd.read_sql(f"SELECT Date date, Open, High, Low, Close, Volume, Volatility, Turnover, symbol FROM daily ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})\
                .drop_duplicates(subset=['date', 'symbol'])
        recommends = pd.read_sql(f"SELECT Date date, symbol, Firm, new_grade, prev_grade, Action from recommendations ORDER BY Date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        arts =pd.read_sql("SELECT date, symbol, publisher, pos_sent, neu_sent, neg_sent, comp_sent FROM articles ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        crypt_arts = pd.read_sql("SELECT date, symbol, publisher,pos_sent, neu_sent, neg_sent, comp_sent  FROM news_sentiment ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        articles = pd.concat([arts, crypt_arts], axis=0, ignore_index=True)
        comments = pd.read_sql(f"SELECT DATE(timestamp) date, channel, symbols, pos_sent, neu_sent, neg_sent, comp_sent from symbol_comments ORDER BY timestamp", parse_dates={'date': '%Y-%m-%d'}, con=con)
        comments.loc[:, "symbols"] = comments.symbols.apply(lambda x: x.replace('BTC', 'BTC-USD'))
        companies = tuple(port.symbol.unique())
        c_data = pd.read_sql(f"SELECT * from mentions WHERE symbol IN {companies}", con=con, index_col='pk')

In [1175]:
symbols_re = re.compile(r"\[|\]|\'|\'")
last_index = comments.index.max()

In [1165]:
# decompose for single symbol
for i, row in comments.iterrows():
    symbols = re.sub(symbols_re, "", row.symbols)
    symbols = symbols.split(',')
    for sym in symbols:
        last_index+=1
        comments.loc[last_index, ["symbols"]] = sym
        comments.loc[last_index, ["comment_index"]] = i
        comments.loc[last_index, ["date", "channel", "pos_sent", "neu_sent", "neg_sent", "comp_sent"]] = row.date, row.channel, row.pos_sent, row.neu_sent,  row.neg_sent, row.comp_sent


In [1195]:
print(time.perf_counter())

time.perf_counter()

75420.9830585


75421.0021197

In [ ]:
comments.assign(sym = lambda x: x.symbols.apply(lambda x: re.sub(symbols_re, '', x)).apply(str.split, sep=',')).explode('sym').reset_index

In [1166]:
comments = comments[lambda x:~( x.comment_index.isnull())]
comments = comments[lambda x: x.symbols.isin(companies)]

In [1167]:
recommendsDict = {"Very Bearish": 1, "Bearish": 2, "Neutral": 3, "Bullish": 4, "Very Bullish": 5}

In [1168]:
recommends=recommends.assign(new_sent = lambda x: x.new_grade.apply(lambda g: recommendsDict[g]))\
    .assign(prev_sent = lambda x: x.prev_grade.apply(lambda g: recommendsDict[g]))

In [1169]:
port.columns

Index(['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Volatility',
       'Turnover', 'symbol'],
      dtype='object')

In [ ]:
def create_daily_data(ticker):
    tick = yf.ticker.Ticker(ticker)
    historical_data = tick.history("1mo")
    outstanding = tick.info.get("sharesOutstanding")
    if outstanding == None:
        outstanding = 1
    daily_close = historical_data["Close"]
    pct_change = daily_close.pct_change().fillna(0)
    periods = 2
    # calc volatility
    vola = (pct_change.rolling(periods).std() * np.sqrt(periods)).fillna(0)
    historical_data = historical_data.assign(Volatility = vola)
    historical_data = historical_data.assign(Turnover = lambda x: x.Volume / outstanding)
    historical_data = historical_data.assign(symbol = ticker)
    return historical_data.reset_index().round({"Volatility": 6, "Turnover": 6})

In [1103]:
# full_daily_data = pd.DataFrame()
# for i, l in enumerate(companies):
#     new_df = create_daily_data(l)
#     full_daily_data = pd.concat([full_daily_data, new_df], axis=0, ignore_index=True)
#     if i % 5 == 0:
#         print(f"Finished with {i}")

In [1104]:
# _22 = full_daily_data[lambda x: x.Date > dt.datetime(2021, 12, 31)]
# _22 = _22.rename(columns={"Stock Splits": "Stock_Splits"})
# _22 = _22.loc[:, port.drop('pk', axis=1).columns]
# new_port = pd.concat([port.drop('pk', axis=1), _22], axis=0, ignore_index=True)

In [1170]:
# take aggregations over wanted frequency; make buy decisions based off of the frequency of data points and sentiments
# return port with new information: shares and cost * shares
class EAT():
    def __init__(self, portfolio, articles, comments, recs, start, end):
        self.portfolio = portfolio.copy(deep=True)
        self.postions = []
        self.start = start
        self.end = end
        self.articles = articles[lambda x: (x.date >= start) & (x.date <= end)]
        self.comments =  comments[lambda x: (x.date >= start) & (x.date <= end)]
        self.recs = recs[lambda x: (x.date >= start) & (x.date <= end)]

        self.aggs = {}

    def aggregate(self):
        articles_agg = self.articles.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol'])\
            .agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'News')
        comments_agg = self.comments.groupby([pd.Grouper(key="date", freq="1Y"), 'symbols'])\
            .agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'Chats')
        recommends_agg = self.recs.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol'])\
            .agg({'new_sent': ['mean'], 'prev_sent': ['mean', 'count']}).assign(type=lambda x: 'Analysts')
        recommends_agg = recommends_agg.reset_index()
        comments_agg = comments_agg.reset_index()
        articles_agg = articles_agg.reset_index()
        recommends_agg.columns = recommends_agg.columns.droplevel(1)
        comments_agg.columns = comments_agg.columns.droplevel(1)
        articles_agg.columns = articles_agg.columns.droplevel(1)
        
        articles_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']
        comments_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']
        recommends_agg.columns = ['date', 'symbol', 'new_sent', 'prev_sent', 'counts', 'type']
        # comments_agg=comments_agg.assign(date = lambda x: x.date.apply(lambda x: x.date))
        self.aggs['recommendations'] = recommends_agg
        self.aggs['articles'] = articles_agg
        self.aggs['comments'] = comments_agg
        return None 


    def tradeSents(self, agg, label, min_samples, min_comp_sent, shares):
        # add action, shares, cost
        returns = self.aggs[agg][lambda x: (x.date >= self.start) & (x[label] >= min_comp_sent) & (x.counts >= min_samples)]
        # query portfolio for first cost add columns
        indexes = pd.Int64Index([])
        for date, sym in returns.loc[:, ['date', 'symbol']].values:
            # ns = returns[lambda x: x.date == date].shape[0]
            if sym not in self.postions:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.date > date) & (x.symbol == sym) & (x.date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares
            else:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.date > date) & (x.symbol == sym) & (x.date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares * self.postions.count(sym)
            
            i = returns[lambda x: (x.date == date) & (x.symbol == sym)].index
            if not indexes.empty:
                returns.loc[i, 'cost'] = shares * self.portfolio.loc[indexes[0], "Close"]
                returns.loc[i, 'returns'] = shares * self.portfolio.loc[indexes[-1], "Close"]
            else:
                indexes = self.portfolio[lambda x: (x.symbol == sym)].index
                returns.loc[i, 'cost'] = shares * self.portfolio.loc[indexes[-1], "Open"]
                returns.loc[i, 'returns'] = shares * self.portfolio.loc[indexes[-1], "Close"]

        return self.portfolio#returns
        


In [1171]:
eat = EAT(port, articles, comments, recommends, dt.datetime(2018, 1, 1), dt.datetime(2022, 1, 30))

In [1172]:
eat.aggregate()

In [1173]:
eat.tradeSents("comments", "comp_sent", min_samples=1, min_comp_sent=0.15, shares=10)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,date,Open,High,Low,Close,Volume,Volatility,Turnover,symbol,shares
0,2017-01-03,42.972000,44.066002,42.192001,43.397999,29616500,0.000000,2.950000e-02,TSLA,NaN
1,2017-01-03,11.420000,11.650000,11.020000,11.430000,55182000,0.000000,4.570000e-02,AMD,NaN
2,2017-01-03,27.250112,27.374832,27.005378,27.332474,115127600,0.000000,7.000000e-03,AAPL,NaN
3,2017-01-03,20.639999,21.840000,20.532000,21.360001,73033,0.000000,4.000000e-04,ACB,NaN
4,2017-01-03,89.000000,89.000000,88.080002,88.599998,8789400,0.000000,3.200000e-03,BABA,NaN
...,...,...,...,...,...,...,...,...,...,...
98728,2022-01-28,24.003000,25.709999,22.809999,25.639999,21496600,0.101186,7.133600e-02,PTON,NaN
98729,2022-01-28,20.660000,21.320000,19.309999,20.900000,96497500,0.058454,6.066900e-02,NIO,NaN
98730,2022-01-28,1.850000,1.870000,1.750000,1.780000,2441000,0.035781,2.085600e-02,SOLO,NaN
98731,2022-01-29,37780.714844,38576.261719,37406.472656,38138.179688,17194183075,0.008032,1.719418e+10,BTC-USD,NaN


In [956]:
ret = eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [957]:
ret = eat.tradeSents("recommendations", "new_sent", 25, 4, 10)

In [958]:
# eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)
# eat.tradeSents("recommendations", "new_sent", 25, 4, 10)

In [1152]:
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

,pos_sent,neg_sent,neu_sent,comp_sent,counts,cost,returns,r_pct
date,,,,,,,,
2019-12-31,5.265895,0.623934,15.110028,8.133532,93,42329.746435,67754.449959,0.600634
2020-12-31,4.164832,0.921737,24.733701,5.878339,1405,330494.633604,502545.053391,0.520585


In [1153]:
ret = eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,pos_sent,neg_sent,neu_sent,comp_sent,counts,cost,returns,r_pct
date,,,,,,,,
2020-12-31,1.886672,0.725528,19.387871,12.806051,6245,363631.928234,537816.954250,0.479015
2021-12-31,2.635620,0.961372,26.403193,18.722140,9305,90168.539557,73207.400317,-0.188105


In [1157]:
ret = eat.tradeSents("recommendations", "new_sent", 25, 4, 10)
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,new_sent,prev_sent,counts,cost,returns,r_pct
date,,,,,,
2018-12-31,29.418501,29.140315,280,21844.340649,27577.331295,0.262447
2019-12-31,33.888142,33.519803,355,30153.087006,47727.473297,0.582839
2020-12-31,67.845811,67.276012,993,57397.397575,62125.701008,0.082378
2021-12-31,58.988827,58.416372,694,67120.639572,56591.000690,-0.156876


In [808]:
with sql.connect('../data/interim/companies.db') as con:
    new_port.to_sql('daily', con=con, index=True, index_label="pk", if_exists='replace')